Para esse segundo lab, vamos usar OO para criar nossos objetos e encapsulá-los de maneira mais eficaz


In [1]:
import pandas as pd
import nltk as nltk
import operator as operator
import math as math

In [2]:
data_depois = pd.read_csv("./Data/estadao_noticias_eleicao.csv")

In [3]:
num_loop = data_depois.shape[0]

In [4]:
#tratando os NaN
data_depois = data_depois.fillna('')
data_depois.head()

,timestamp,titulo,subTitulo,conteudo,url,idNoticia
0,2014-12-31T00:00:00Z,PT espera 30 mil pessoas em festa na Esplanada,Objetivo é demonstrar apoio popular a Dilma e ...,BRASÍLIA - Após o desgaste provocado com o lan...,"http://politica.estadao.com.br/noticias/geral,...",1
1,2014-12-31T00:00:00Z,Alckmin toma posse de olho no Planalto,Governador reeleito tenta amarrar tucanos paul...,"Reeleito em outubro, o governador tucano Geral...","http://politica.estadao.com.br/noticias/geral,...",2
2,2014-12-31T00:00:00Z,Seis obstáculos e desafios do segundo mandato ...,"Em meio a escândalo de corrupção, presidente t...",1. Rearranjo das contas A nova equipe econôm...,"http://politica.estadao.com.br/noticias/geral,...",3
3,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",4
4,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",5


In [83]:
#dicionario das palavras
dict_palavras = dict()

In [28]:
def merge(first_tuple,second_tuple):
    """
    function that will merge two tuples
    
    Args
    ----------
    first_tuple: Tuple 
        first tuple we wont to merge
    second_tuple: Tuple 
        second tuple we wont to merge
    ----------
    
    Returns
    ----------
    merge_tuple: Tuple
        return the merge between first_tuple and second_tuple
    ----------
    """
    merged_tuple = [] 
    for y in range(num_loop):
        merged_tuple.append((first_tuple[y][0],first_tuple[y][1] + second_tuple[y][1]))
    return merged_tuple

In [88]:
def merge_binary(first_tuple,second_tuple):
    """
    function that will merge two binary tuples
    
    Args
    ----------
    first_tuple: Tuple 
        first binary tuple we wont to merge
    second_tuple: Tuple 
        second binary tuple we wont to merge
    ----------
    
    Returns
    ----------
    merge_tuple: Tuple
        return the merge between first_tuple and second_tuple, adding both binary numbers
    ----------
    """
    merged_binary_tuple = [] 
    for y in range(num_loop):
        merged_binary_tuple.append((first_tuple[y][0],first_tuple[y][2] + second_tuple[y][2]))
    return merged_binary_tuple

In [84]:
def search_query(searched_word):
    """
    function that will search for words equals "searched_word" in documents
    
    Args
    ---------
    searched_word: String
        unique word that gonna be searched in documents
    ---------
    
    Returns
    ---------
    dict_palavras[searched_word]: Array
            return an array with all documents,frequency of occurrence and a binary term (that says 
            if the world is in that document or not) about that searched_word by document
    ---------
    """
    if(searched_word in dict_palavras):
        return dict_palavras[searched_word]
    else:
        dict_palavras[searched_word] = []
        for i in range(num_loop):
            tf = 0
            titulo = data_depois.loc[i,'titulo']
            sub_titulo = data_depois.loc[i,'subTitulo']
            conteudo = data_depois.loc[i,'conteudo']
            new_world = titulo + " " + sub_titulo + " " + conteudo
            new_world_nova = nltk.word_tokenize(new_world.lower())
            for palavra in new_world_nova:
                if(palavra == searched_word):
                    tf = tf + 1
            if(tf > 0):
                dict_palavras[searched_word].append((data_depois.loc[i,'idNoticia'],tf,1))
            else:
                dict_palavras[searched_word].append((data_depois.loc[i,'idNoticia'],tf,0))
    return dict_palavras[searched_word]

In [85]:
len(search_query('segundo'))

8716

In [86]:
def simplest_vsm(query):
    """
    
    
    """

    query_splited = nltk.word_tokenize(query)
    i = 0
    while(i < len(query_splited )-1):
        final_result = merge_binary(search_query(query_splited[i]),search_query(query_splited[i+1]))
        i+=1
    final_result.sort(key = operator.itemgetter(1),reverse = True)
    return final_result

    

In [89]:
a = simplest_vsm('segundo turno')
#TOP5
print(a[:5])

[(1, 2), (7, 2), (13, 2), (26, 2), (69, 2)]


In [35]:
def tf_vsm(query):
    """
    function that will search for query's words in documents and return a array of 
    
    tuple sorted by te frequency about these words
    
    Args
    ----------
    query: String
        this query come from the user and will serve for do the search, looking for these words on the documents
    ----------
    
    Returns
    ---------
    final_resut: Array
        this array contain a sorted documents by the frequency of these query's words
    ---------
    """
    query_splited = nltk.word_tokenize(query)
    i = 0
    while(i < len(query_splited )-1):
        final_result = merge(search_query(query_splited[i]),search_query(query_splited[i+1]))
        i+=1
    final_result.sort(key = operator.itemgetter(1),reverse = True)
    return final_result



In [37]:
b = tf_vsm('segundo turno')
print(b[:5])

[(2744, 43), (7, 28), (2112, 26), (7672, 26), (2388, 24)]


In [46]:
#Modelo IDF
#TODO working just for len(query) equals two


def idf_vsm(query):
    """
    """
    query_splited = nltk.word_tokenize(query)
    first_tuple = []
    second_tuple = []
    idf1 = 0
    idf2 = 0
    first_idf = 0
    second_idf = 0 
    for i in  search_query(query_splited[0]):
        if(i[1] > 0):
            first_idf += 1
    print (first_idf)
    idf1 = math.log((num_loop+1)/first_idf)
    print(idf1)
    print(query_splited[0])
    
    for j in search_query(query_splited[1]):
        if(j[1] > 0):
            second_idf +=1
    print(second_idf)
    idf2 = math.log((num_loop + 1)/second_idf)
    print(idf2)
    print(query_splited[1])
    
    
      
    #a = merge_binary(first_tuple,second_tuple)
    #a.sort(key = operator.itemgetter(1),reverse = True)
    #return a


    


In [47]:
idf_vsm('segundo turno')

4161
0.7395197120286349
segundo
1441
1.799937825012054
turno
